In [ ]:
!pip install pgmpy #Probabilistic graph model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 47.5 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD #For showing table probability distribution
from pgmpy.inference import VariableElimination #For calculating joint probability and various other probability

In [ ]:
#Structure of the Bayesian Network
model_structure = DiscreteBayesianNetwork([
    ('Burglary','Alarm'),  #(Parent,child)
    ('Earthquake','Alarm'),
    ('Alarm','JohnCalls'),
    ('Alarm','MerryCalls')
])

In [ ]:
#CPDs (Conditional Probability Distribution)
cpd_burglary = TabularCPD(variable='Burglary',variable_card=2,values=[[.999],[.001]])  #values = event not happened, event happened -> passed in form of list
#The first row corresponds to Burglary = 0 i.e, Burglary not occurred
#The second row corresponds to Burglary  = 1 i.e, Burglary occurred
cpd_earthquake = TabularCPD(variable='Earthquake',variable_card=2,values=[[.998],[.002]])

In [ ]:
print(cpd_burglary)

+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+


In [ ]:
print(cpd_earthquake)

+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_alarm = TabularCPD(
    variable='Alarm',
    variable_card=2,
    values = [
        [0.999, 0.71, 0.06, 0.05], #P(Alarm = 0 | Burglary, Earthquake -> 0 0,0 1,1 0,1 1)
        [0.001, 0.29, 0.94, 0.95]  #P(Alarm = 1 | Burglary, Earthquake -> 0 0,0 1,1 0,1 1)
    ],
    evidence = ['Burglary','Earthquake'],
    evidence_card = [2,2]
)
print(cpd_alarm)

+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


In [ ]:
cpd_johncalls = TabularCPD(
    variable='JohnCalls',
    variable_card=2, # variable card means how many values event can take i.e, JohnCalls will happened or not happened
    values = [
        [0.95, 0.1],
        [0.05, 0.9]
    ],
    evidence = ['Alarm'],
    evidence_card = [2]
)

In [ ]:
cpd_merrycalls = TabularCPD(
    variable='MerryCalls',
    variable_card=2, # variable card means how many values event can take i.e, MerryCalls will happened or not happened
    values = [
        [0.99, 0.3],
        [0.01, 0.7]
    ],
    evidence = ['Alarm'],
    evidence_card = [2]
)

In [ ]:
print(cpd_johncalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+----------+


In [ ]:
print(cpd_merrycalls)

+---------------+----------+----------+
| Alarm         | Alarm(0) | Alarm(1) |
+---------------+----------+----------+
| MerryCalls(0) | 0.99     | 0.3      |
+---------------+----------+----------+
| MerryCalls(1) | 0.01     | 0.7      |
+---------------+----------+----------+


In [ ]:
model_structure.add_cpds(cpd_burglary,cpd_earthquake,cpd_alarm,cpd_johncalls,cpd_merrycalls)

In [ ]:
model_structure.check_model() # For checking whether model is working on not i.e, True-> working, False -> Not working

True

In [ ]:
inference = VariableElimination(model_structure)

In [ ]:
query_result = inference.query(variables=['Burglary'],evidence={'JohnCalls':1,'MerryCalls':1})
print(query_result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


# **Query 1 : P(JohnCalls, MerryCalls, Alarm | Burglary = 0, Earthquake = 0)**

In [ ]:
evidence = {
    'Burglary':0,
    'Earthquake':0
}
result1 = inference.query(variables=['JohnCalls','MerryCalls','Alarm'],evidence=evidence)
print(result1)


+--------------+---------------+----------+-----------------------------------+
| JohnCalls    | MerryCalls    | Alarm    |   phi(JohnCalls,MerryCalls,Alarm) |
+==============+===============+==========+===================================+
| JohnCalls(0) | MerryCalls(0) | Alarm(0) |                            0.9396 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(0) | Alarm(1) |                            0.0000 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(1) | Alarm(0) |                            0.0095 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(1) | Alarm(1) |                            0.0001 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(1) | MerryCalls(0) | Alarm(0) |                            0.0495 |
+--------------+---------------+--------

In [ ]:
probability1 = result1.values[1][1][1]
print("Probability of Alarm Ringing when JohnCalls = True, MerryCalls = True, Burglary = False, Earthquake = False : ",probability1)


Probability of Alarm Ringing when JohnCalls = True, MerryCalls = True, Burglary = False, Earthquake = False :  0.0006300000000000001


# **Query 2 : P(John Calls)**

# **P(j) = P(j|a)P(a) + P(j|-a)P(-a)**

In [ ]:
result2 = inference.query(variables=['JohnCalls'])
print(result2)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
probability2 = result2.values[1]
print("Probability of John Calling : ",probability2)

Probability of John Calling :  0.0521389757


# **Query 3 : P(Burglary | JohnCalls, MerryCalls)**

In [ ]:
evidence = {
    'JohnCalls':1,
    'MerryCalls':1
}
result3 = inference.query(variables=['Burglary'],evidence=evidence)
print(result3)


+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
probability3 = result3.values[1]
print("Probability of Burglary when JohnCalls = True, MerryCalls = True : ",probability3)


Probability of Burglary when JohnCalls = True, MerryCalls = True :  0.284171835364393


# **Query 4 : P(Earthquake | JohnCalls = 0, MerryCalls = 1)**

In [ ]:
evidence = {
    'JohnCalls':0,
    'MerryCalls':1
}
result4 = inference.query(variables=['Earthquake'],evidence=evidence)
print(result4)


+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9944 |
+---------------+-------------------+
| Earthquake(1) |            0.0056 |
+---------------+-------------------+


In [ ]:
probability4 = result4.values[1]
print("Probability of Earthquake when JohnCalls = False, MerryCalls = True : ",probability4)

Probability of Earthquake when JohnCalls = False, MerryCalls = True :  0.005612151520557884


# **Query 5 : P(MerryCalls = False)**

# **P(-j) = P(-j|a)P(a) + P(-j|-a)P(-a)**

In [ ]:
result5 = inference.query(variables=['MerryCalls'])
print(result5)


+---------------+-------------------+
| MerryCalls    |   phi(MerryCalls) |
+===============+===================+
| MerryCalls(0) |            0.9883 |
+---------------+-------------------+
| MerryCalls(1) |            0.0117 |
+---------------+-------------------+


In [ ]:
probability5 = result5.values[0]
print("Probability of Merry Not Calling : ",probability5)


Probability of Merry Not Calling :  0.98826365502
